In [1]:
# Heat budget terms for the different layers
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os

/Users/weiyangbao/miniconda3/envs/py37/lib/python3.7/site-packages/xarray/core/merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [2]:
sd = 15 # sill depth

state01 = xr.open_dataset('/Users/weiyangbao/Documents/Model_outputs/State700/Hs_' + str(format(sd,'03d')) + '.nc')
state02 = xr.open_dataset('/Users/weiyangbao/Documents/Model_outputs/State700/DFx_TH_' + str(format(sd,'03d')) + '.nc')
state03 = xr.open_dataset('/Users/weiyangbao/Documents/Model_outputs/State700/DFrI_TH_' + str(format(sd,'03d')) + '.nc')

state1 = state01.isel(Y=range(1,10), T=range(40,121))
state2 = state02.isel(Y=range(1,10), T=range(39,120))
state3 = state03.isel(Y=range(1,10), T=range(39,120))

Area = np.empty([50, 9]) # Cross Y direction
Area[:10, :] = 100
Area[10:20, :] = 200
Area[20:30, :] = 300
Area[30:40, :] = 800
Area[40:, :] = 1600

T0 = 9 # Initial temperature
f = 3 # Control volume start index near the ice front
s = 28 # Control volume end index at the sill crest
Cp = 4220  # Specific heat content J/kg/C
rho = 1025 # Density

In [3]:
# Read data in T,Z,Y,X
pt = state1.Temp.data

u0 = state1.U.data
u = 0.5 * (u0[:,:,:,:-1] + u0[:,:,:,1:])

w0 = state1.W.data
w = 0.5 * (w0[:,:-1,:,:] + w0[:,1:,:,:])

upt = u * (pt-T0)

dfx0 = state2.DFxE_TH.data
dfx = 0.5 * (dfx0[:,:,:,:-1] + dfx0[:,:,:,1:])

dfz0 = state3.DFrI_TH.data
dfz = 0.5 * (dfz0[:,:-1,:,:] + dfz0[:,1:,:,:])


In [4]:
# Advective flux of potential temperature
ADf = np.sum(Area * upt[:,:,:,f].mean(0)) * Cp * rho 
ADs = -np.sum(Area * upt[:,:,:,s].mean(0)) * Cp * rho 

# Diffusive flux of potential temperature
DFf = np.sum(dfx[:,:,:,f].mean(0)) * Cp * rho
DFs = -np.sum(dfx[:,:,:,s].mean(0)) * Cp * rho 

"{:.2E}".format(ADf), "{:.2E}".format(ADs), "{:.2E}".format(DFf), "{:.2E}".format(DFs), "{:.2E}".format(ADf+DFf+ADs+DFs)

('-2.72E+10', '2.60E+10', '-3.89E+08', '1.80E+09', '1.64E+08')

In [5]:
## Budget terms for different layers

uzx = u.mean(axis=(0,2)) # Z, X
# Time average
wt = w.mean(0) # Z, Y, X
#dfzt = dfz.mean(0)
ptt = pt.mean(0)
dxdy = 200**2

# Find the interface and vertical flux of theta
def Layer(uzx):
    # Layer interface, zero velocity crossing point
    X = uzx.shape[1]
    ADzx = np.empty(X)
    DFzx = np.empty(X)
    itf = np.empty(X, dtype=int)
    ADzx[[0,-1]] = 0
    DFzx[[0,-1]] = 0
    itf[[0,-1]] = 0
    for i in range(1,X-1):
        l = np.where(uzx[:, i] < 0)[-1][0]
        itf[i] = l
        ADzx[i] = np.sum(wt[l,:,i] * (ptt[l,:,i]-T0) * dxdy ) # Vertical advective flux through the interface
        DFzx[i] = np.sum(dfz[:,l,:,i].mean(0)) # Vertical diffusive flux through the interface
    return (itf, ADzx, DFzx)

In [6]:
itf, ADzx, DFzx = Layer(uzx)

ADz = ADzx[f:s+1].sum() 
DFz = DFzx[f:s+1].sum() 


ADf1 = np.sum(Area[:itf[f],:] * upt[:,:itf[f],:,f].mean(0)) 
ADs1 = -np.sum(Area[:itf[s],:] * upt[:,:itf[s],:,s].mean(0)) 
DFf1 = np.sum(dfx[:,:itf[f],:,f].mean(0)) 
DFs1 = -np.sum(dfx[:,:itf[s],:,s].mean(0)) 


ADf2 = np.sum(Area[itf[f]:,:] * upt[:,itf[f]:,:,f].mean(0))
ADs2 = -np.sum(Area[itf[s]:,:] * upt[:,itf[s]:,:,s].mean(0))
DFf2 = np.sum(dfx[:,itf[f]:,:,f].mean(0))
DFs2 = -np.sum(dfx[:,itf[s]:,:,s].mean(0))

ptz = 0.5 * (ptt[itf[s]-1,:,f:s+1] + ptt[itf[s],:,f:s+1])
ADz1 = np.sum(wt[itf[s],:,f:s+1].mean(0) * (ptz-T0) * dxdy)
#ADz2 = np.sum(wt[itf[s],:,f:s+1] * (ptt[itf[s],:,f:s+1]-T0) * dxdy)
DFz1 = np.sum(dfz[:,itf[s],:,f:s+1].mean(0))

#ADf2 = np.sum(Area[itf[f]:,:] * upt[:,itf[f]:,:,f].mean(0))
#ADs2 = ADs - ADs1

ADf1+ADs1, DFf1+DFs1, "{:.2E}".format(ADz*Cp*rho), "{:.2E}".format(DFz*Cp*rho), ADf1+ADs1+DFf1+DFs1+ADz+ DFz

(-789.7588087012991, 42.92323, '4.44E+09', '2.32E+08', 333.52691756282)

In [ ]:
ADf2, ADs2, DFf2, DFs2, ADf2+ADs2+DFf2+DFs2-ADz

In [ ]:
np.sum(w[:,0,:,f:s+1].mean(0) * (pt[:,0,:,f:s+1]-T0).mean(0) * dxdy), 

In [ ]:
np.sum(dfz[:,0,:,f:s+1].mean(0))

In [ ]:
tem = pt.mean(axis=(0,2))
tem[:,27]

In [ ]:
T = upt.shape[0]

ADft = np.empty(T)
ADst = np.empty(T)
DFft = np.empty(T)
DFst = np.empty(T)
for tt in range(T):
    
    ADft[tt] = np.sum(Area * upt[tt,:,:,f]) 
    ADst[tt] = -np.sum(Area * upt[tt,:,:,s]) 

    DFft[tt] = np.sum(dfx[tt,:,:,f])
    DFst[tt] = -np.sum(dfx[tt,:,:,s])

x = np.arange(81)    
plt.plot(x,ADft)
plt.plot(x,-ADst)
#plt.plot(x,DFft)
#plt.plot(x,DFst)